# Secondary Structure Word2Vec Encoder

This demo creates a dataset of sequence segments derived from a non-redundent set. The dataset contains the seuqence segment, the DSSP Q8 and DSSP Q3 code of the center residue in a sequnece segment, and a Word2Vec encoding of the seuqnece segment.

## Imports

In [1]:
from pyspark import SparkConf, SparkContext, SQLContext
from mmtfPyspark.ml import proteinSequenceEncoder
from mmtfPyspark.mappers import structureToPolymerChains
from mmtfPyspark.filters import containsLProteinChain
from mmtfPyspark.datasets import secondaryStructureSegmentExtractor
from mmtfPyspark.webfilters import pisces
from mmtfPyspark.io import MmtfReader
import time

## Configure Spark Context

In [2]:
conf = SparkConf() \
        .setMaster("local[*]") \
        .setAppName("secondaryStructureWord2VecEncodeDemo")
sc = SparkContext(conf = conf)

## Read in, filter and sample Hadoop Sequence Files

In [3]:
path = "../../resources/mmtf_reduced_sample/"

sequenceIdentity = 20
resolution = 2.0
fraction = 0.95
seed = 123

pdb = MmtfReader \
        .read_sequence_file(path, sc) \
        .flatMap(structureToPolymerChains()) \
        .filter(pisces(sequenceIdentity, resolution)) \
        .filter(containsLProteinChain()) \
        .sample(False, fraction, seed)

## Extract Secondary Structure Segments

In [4]:
segmentLength = 11
data = secondaryStructureSegmentExtractor.getDataset(pdb, segmentLength).cache()

## Add Word2Vec encoded feature vector

In [5]:
encoder = proteinSequenceEncoder(data)

n = 2
windowSize = (segmentLength -1) // 2
vectorSize = 50
data = encoder.overlappingNgramWord2VecEncode(n, windowSize, vectorSize)

## Show dataset schema and few rows of data

In [7]:
data.printSchema()
data.show(10, False)

root
 |-- structureChainId: string (nullable = false)
 |-- sequence: string (nullable = false)
 |-- labelQ8: string (nullable = false)
 |-- labelQ3: string (nullable = false)
 |-- ngram: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- features: vector (nullable = true)

+----------------+-----------+-------+-------+----------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Terminate Spark Context

In [8]:
sc.stop()